In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv
/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv
/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv


In [4]:
import os 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor, AutoModelForPreTraining, Trainer, TrainingArguments
from datasets import Dataset
from datasets import Dataset as HFDataset
from datasets import load_dataset
import evaluate
import torch
from torch.utils.data import Dataset, DataLoader
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm

import warnings 
warnings.filterwarnings('ignore')

from huggingface_hub import login
login()

In [5]:
prompt_df = pd.read_csv("hf://datasets/bitext/Bitext-customer-support-llm-chatbot-training-dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv")
prompt_df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [7]:
prompt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26872 entries, 0 to 26871
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   flags        26872 non-null  object
 1   instruction  26872 non-null  object
 2   category     26872 non-null  object
 3   intent       26872 non-null  object
 4   response     26872 non-null  object
dtypes: object(5)
memory usage: 1.0+ MB


In [8]:
pattern = r'\{\{(.*?)\}\}'

special = prompt_df['instruction'].apply(lambda x: re.findall(pattern, x)[0] if re.findall(pattern, x) else None)

special.value_counts()

instruction
Order Number        2907
Account Type        1011
Person Name          887
Account Category     822
Currency Symbol      372
Refund Amount        252
Delivery City        234
Delivery Country     177
Invoice Number         8
Name: count, dtype: int64

In [9]:
class DialogueDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Combine instruction and response with special tokens
        dialogue = f"{row['instruction']} [SEP] {row['response']}"

        # Tokenize the text
        encodings = self.tokenizer(
            dialogue,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )

        # Create labels (same as input_ids for language modeling)
        labels = encodings.input_ids.clone()

        # Convert to tensor format and squeeze extra dimension
        return {
            'input_ids': encodings.input_ids.squeeze(),
            'attention_mask': encodings.attention_mask.squeeze(),
            'labels': labels.squeeze()
        }

def prepare_data(df, tokenizer, train_size=0.9, max_length=128):
    # Convert DataFrame to Hugging Face Dataset
    dataset_dict = {
        'instruction': df['instruction'].tolist(),
        'response': df['response'].tolist()
    }
    dataset = HFDataset.from_dict(dataset_dict)

    # Split dataset
    dataset = dataset.train_test_split(train_size=train_size)

    def tokenize_function(examples):
        # Combine instruction and response
        dialogues = [f"{instr} {tokenizer.eos_token} {resp}"
                    for instr, resp in zip(examples['instruction'], examples['response'])]

        # Tokenize
        tokenized = tokenizer(
            dialogues,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors='pt'
        )

        # Create labels (shift input_ids right by one position)
        labels = tokenized['input_ids'].clone()
        labels[labels == tokenizer.pad_token_id] = -100  # Ignore padding tokens in loss

        return {
            'input_ids': tokenized['input_ids'],
            'attention_mask': tokenized['attention_mask'],
            'labels': labels
        }

    # Tokenize datasets
    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=dataset['train'].column_names
    )

    return tokenized_dataset


In [11]:
def train_model(df, model_name="microsoft/DialoGPT-small", output_dir="./fine_tuned_dialo-gpt"):
    # Initialize tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Ensure padding token is set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id

    # Prepare dataset
    tokenized_dataset = prepare_data(df, tokenizer)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=100,
        eval_strategy="steps",
        eval_steps=500,
        save_strategy="steps",
        save_steps=1000,
        load_best_model_at_end=True,
        remove_unused_columns=False
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset['train'],
        eval_dataset=tokenized_dataset['test'],
    )

    # Train the model
    trainer.train()

    return model, tokenizer

In [12]:
# Train the modelb
model, tokenizer = train_model(prompt_df)

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

...

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,1.512800,1.262065
1000,1.143400,1.010988
1500,1.027900,0.928990
2000,0.965500,0.884815
2500,0.919500,0.858718
3000,0.907300,0.838395
3500,0.876900,0.825499
4000,0.850400,0.818459
4500,0.858000,0.813984


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


In [13]:
# Push model and tokenizer directly to Hugging Face Hub
model.push_to_hub("khalednabawi11/fine_tuned_dialo-gpt")
tokenizer.push_to_hub("khalednabawi11/fine_tuned_dialo-gpt")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/khalednabawi11/fine_tuned_dialo-gpt/commit/f974b2318c3976fc19e023ba075e3abb2d9aa555', commit_message='Upload tokenizer', commit_description='', oid='f974b2318c3976fc19e023ba075e3abb2d9aa555', pr_url=None, repo_url=RepoUrl('https://huggingface.co/khalednabawi11/fine_tuned_dialo-gpt', endpoint='https://huggingface.co', repo_type='model', repo_id='khalednabawi11/fine_tuned_dialo-gpt'), pr_revision=None, pr_num=None)

In [16]:
def generate_response(prompt, max_length=150):
    # Set device (GPU if available, else CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Move model to the same device
    model.to(device)

    # Clean up the prompt - replace placeholder with actual value
    clean_prompt = prompt.replace("{{Order Number}}", "").strip()

    # Tokenize input and move inputs to the correct device
    inputs = tokenizer(
        clean_prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        return_attention_mask=True
    ).to(device)

    # Generate outputs
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        do_sample=True,
        temperature=0.6,  # Slightly lower temperature for more focused responses
        no_repeat_ngram_size=3,
        num_beams=3,      # Add beam search
        early_stopping=True,
        top_p=0.92,
        top_k=50,
        repetition_penalty=1.2  # Prevent repetition
    )

    # Decode the response and move back to CPU
    response = tokenizer.decode(outputs[0].to('cpu'), skip_special_tokens=True)

    # Clean up the response
    if clean_prompt in response:
        response = response[len(clean_prompt):].strip()

    # Format numbered lists properly
    lines = response.split('\n')
    formatted_lines = []
    for line in lines:
        if any(line.startswith(str(i)) for i in range(10)):
            formatted_lines.append('\n' + line)
        else:
            formatted_lines.append(line)

    response = ' '.join(formatted_lines)

    return response

# Test the improved generation
prompt = "How to pay for the product? {{12345}} "
response = generate_response(prompt)
print("Prompt:", prompt)
print("\nResponse:", response)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: How to pay for the product? {{12345}} 

Response: is it possible to place an order?    would like to know more about canceling purchase {{Order Number}}, how can I do it?   To cancel your purchase, please follow these steps:  
1. Sign into Your Account: Access our platform by signing in to your {{Online Company Portal Info}}. 
2. Navigate to Your Orders: Once you're logged in, navigate to the '{{Online Order Interaction}}' section. 
3. Locate Your Order: Look for the purchase associated with the order number {{Purchase Number}} and click on it for more details. 
4. If you encounter any difficulties or difficulties, please don


In [21]:
def generate_response(prompt, max_length=150):
    # Set device (GPU if available, else CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Move model to the appropriate device
    model.to(device)

    # Clean up the prompt - replace placeholder with an empty value or use a template
    clean_prompt = prompt.replace("{{Order Number}}", "12345").strip()  # Using a dummy order number for better flow

    # Tokenize input and move inputs to the correct device
    inputs = tokenizer(
        clean_prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        return_attention_mask=True
    ).to(device)

    # Generate outputs with improvements
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        do_sample=True,                      
        temperature=0.7,                     
        num_beams=5,                          
        no_repeat_ngram_size=2,               
        early_stopping=True,                  
        top_p=0.9,                            
        top_k=50,                             
        repetition_penalty=1.2,               
        pad_token_id=tokenizer.pad_token_id   
    )

    # Decode the response, moving back to CPU and ensuring proper token cleanup
    response = tokenizer.decode(outputs[0].to('cpu'), skip_special_tokens=True)

    # Clean up the response by trimming redundant parts
    if clean_prompt in response:
        response = response[len(clean_prompt):].strip()

    # Additional formatting - clean up lists and add spacing for structured responses
    lines = response.split('\n')
    formatted_lines = []
    for line in lines:
        # Handle numbered lists formatting
        if any(line.lstrip().startswith(str(i)) for i in range(10)):  # Handle number lists 1-9
            formatted_lines.append('\n' + line.lstrip())  # Add space before numbered lines
        else:
            formatted_lines.append(line.lstrip())

    response = ' '.join(formatted_lines).strip()

    return response

# Test the improved generation
prompt = "What is the delivery city for my order?"
response = generate_response(prompt)
print("Prompt:", prompt)
print("\nResponse:", response)

Prompt: What is the delivery city for my order?

Response: {{Delivery City}}, I'm happy to help! We offer delivery services to various locations, including the United States, Canada, and Mexico. Whether you're looking for standard shipping or expedited shipping, we've got you covered. If you have any specific questions or need further assistance, feel free to reach out to our customer support team. We're here to ensure your order reaches you safely and efficiently. How else may I assist you today? Your satisfaction is our top priority! Is there anything else I can help you with? Let's work together to make your delivery experience seamless We appreciate your patience and look forward to resolving this matter for you. Please let me know if there's anything


In [23]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the fine-tuned model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("khalednabawi11/fine_tuned_dialo-gpt")
model = AutoModelForCausalLM.from_pretrained("khalednabawi11/fine_tuned_dialo-gpt")

# Function to generate a response
def generate_response(user_input, conversation_history=None):
    if conversation_history is None:
        conversation_history = ""
        
    # Combine conversation history and new user input
    input_text = conversation_history + f"User: {user_input}\nBot:"

    # Tokenize the input
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    
    # Generate response
    outputs = model.generate(inputs, max_length=150, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    
    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the bot's reply from the response
    bot_reply = response.split("Bot:")[-1].strip()
    
    return bot_reply, response

# Example usage
user_input = "What is the delivery city for my order?"
bot_response, full_response = generate_response(user_input)

print("Bot Response:", bot_response)

Bot Response: We understand that you would like to know the delivery city for your order. To provide you with accurate information, could you please provide us with the {{Delivery City}} or {{Delivery City}}? With this information, we will be able to assist you further in determining the delivery location. Your satisfaction is our top priority, and we are here to ensure a smooth and seamless delivery experience for you. Please let us know the specific city or region you are looking for, and we will be more than happy to provide you with the delivery details. Your satisfaction is our top priority


In [31]:
prompt_df['input_ids'] = prompt_df['instruction'].apply(lambda x: tokenizer.encode(x, return_tensors='pt').squeeze(0))
prompt_df['target_ids'] = prompt_df['response'].apply(lambda x: tokenizer.encode(x, return_tensors='pt').squeeze(0))

# Sample the dataset
test_dataset = prompt_df.sample(2000)

def evaluate_model(model, tokenizer, test_dataset, batch_size=32):
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    smooth = SmoothingFunction().method1

    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    bleu_scores = []

    model.eval()
    num_examples = len(test_dataset)
    num_batches = (num_examples + batch_size - 1) // batch_size

    with torch.no_grad():
        for i in tqdm(range(num_batches)):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, num_examples)
            batch = test_dataset[start_idx:end_idx]

            input_texts = []
            reference_texts = []

            # Access the input_ids and target_ids from the batch
            for input_ids, target_ids in zip(batch['input_ids'], batch['target_ids']):
                input_texts.append(tokenizer.decode(input_ids, skip_special_tokens=False))
                reference_texts.append(tokenizer.decode(target_ids, skip_special_tokens=False))

            # Generate predictions
            inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)
            outputs = model.generate(
                inputs["input_ids"].to(model.device),
                max_length=100,
                num_return_sequences=1,
                temperature=0.7,
                no_repeat_ngram_size=2,
                pad_token_id=tokenizer.pad_token_id
            )

            predicted_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

            # Calculate scores for each example in batch
            for pred, ref in zip(predicted_texts, reference_texts):
                # ROUGE scores
                rouge_scores = rouge.score(pred, ref)
                rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
                rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
                rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

                # BLEU score
                pred_tokens = pred.split()
                ref_tokens = ref.split()
                if len(pred_tokens) > 0 and len(ref_tokens) > 0:
                    bleu = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smooth)
                    bleu_scores.append(bleu)

    results = {
        'rouge1': np.mean(rouge1_scores),
        'rouge2': np.mean(rouge2_scores),
        'rougeL': np.mean(rougeL_scores),
        'bleu': np.mean(bleu_scores)
    }

    results.update({
        'rouge1_std': np.std(rouge1_scores),
        'rouge2_std': np.std(rouge2_scores),
        'rougeL_std': np.std(rougeL_scores),
        'bleu_std': np.std(bleu_scores)
    })

    return results


In [32]:
def print_evaluation_results(results):
    """Pretty print the evaluation results"""
    print("\nEvaluation Results:")
    print("=" * 50)
    metrics = [
        ('ROUGE-1', 'rouge1'),
        ('ROUGE-2', 'rouge2'),
        ('ROUGE-L', 'rougeL'),
        ('BLEU', 'bleu')
    ]

    for metric_name, metric_key in metrics:
        mean = results[metric_key]
        std = results[f'{metric_key}_std']
        print(f"{metric_name:8} : {mean:.4f} ± {std:.4f}")

# Example usage

results = evaluate_model(model, tokenizer, test_dataset)
print_evaluation_results(results)

100%|██████████| 63/63 [11:19<00:00, 10.78s/it]


Evaluation Results:
ROUGE-1  : 0.4458 ± 0.1324
ROUGE-2  : 0.2005 ± 0.1188
ROUGE-L  : 0.2959 ± 0.1089
BLEU     : 0.0950 ± 0.0912
